In [1]:
import re
import subprocess

import dxdata
import dxpy
import pandas as pd
import pyspark

from pyspark.sql import functions as F
from pyspark.conf import SparkConf
from pyspark.sql.types import StringType

from pathlib import Path

Path("../tmp").resolve().mkdir(parents=True, exist_ok=True)

In [2]:
conf = SparkConf()
conf.set("autoBroadcastJoinThreshold", -1)

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

dispensed_database_name = dxpy.find_one_data_object(
    classname="database", name="app*", folder="/", name_mode="glob", describe=True
)["describe"]["name"]
dispensed_dataset_id = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)["id"]

spark.sql("USE " + dispensed_database_name)

dataset = dxdata.load_dataset(id=dispensed_dataset_id)
participant = dataset["participant"]


def get_columns_to_keep(df, threshold=200) -> list:
    """
    This function drops all columns which contain more null values than threshold
    :param df: A PySpark DataFrame
    """
    null_counts = (
        df.select([F.count(F.when(~F.col(c).isNull(), c)).alias(c) for c in df.columns])
        .collect()[0]
        .asDict()
    )
    to_keep = [k for k, v in null_counts.items() if v > threshold]
    # df = df.select(*to_keep)

    return to_keep


def new_names(s: str) -> str:
    """
    Fixes a column header for PHESANT use
    """
    s = s.replace("p", "x").replace("i", "")

    if bool(re.search("_\d$", s)):
        s += "_0"
    else:
        s += "_0_0"
    return s

In [9]:
first_occurences = list(
    participant.find_fields(
        lambda f: bool(re.match("^Date [F]\d{2} first reported", f.title))
    )
)

age_sex = "|".join(["31", "21022"])
age_sex_fields = list(
    participant.find_fields(lambda f: bool(re.match(f"^p({age_sex})$", f.name)))
)

psych = "|".join(
    [
        "2090",
        "2100",
        "20126",
    ]
)

psych_fields = list(
    participant.find_fields(lambda f: bool(re.match(f"^p({psych})\D", f.name)))
)

field_names = (
    ["eid"]
    + [f.name for f in age_sex_fields]
    + [f.name for f in first_occurences]
    + [f.name for f in psych_fields]
)
df = participant.retrieve_fields(names=field_names, engine=dxdata.connect())

[dxdata.dataset.dataset] WARNING: Dataframe schema name(s): ['participant_0001$eid', 'participant_0001$p31', 'participant_0042$p21022', 'participant_0085$p130836', 'participant_0085$p130838', 'participant_0085$p130840', 'participant_0085$p130842', 'participant_0085$p130844', 'participant_0085$p130846', 'participant_0085$p130848', 'participant_0085$p130850', 'participant_0086$p130852', 'participant_0086$p130854', 'participant_0086$p130856', 'participant_0086$p130858', 'participant_0086$p130860', 'participant_0086$p130862', 'participant_0086$p130864', 'participant_0086$p130866', 'participant_0086$p130868', 'participant_0086$p130870', 'participant_0086$p130872', 'participant_0086$p130874', 'participant_0086$p130876', 'participant_0086$p130878', 'participant_0086$p130880', 'participant_0086$p130882', 'participant_0086$p130884', 'participant_0086$p130886', 'participant_0086$p130888', 'participant_0086$p130890', 'participant_0086$p130892', 'participant_0086$p130894', 'participant_0086$p13089

In [10]:
min_cases = 500

print(f"Number of columns: {len(df.columns)}")
to_keep = get_columns_to_keep(df, min_cases)

#to_keep.insert(1, to_keep[11])
#to_keep.insert(2, to_keep[11])
#to_keep.pop(12)
#to_keep.pop(12)

df = df.select(*to_keep)

print(f"Number of columns with at least {min_cases} cases: {len(df.columns)}")

Number of columns: 88
Number of columns with at least 500 cases: 39


In [11]:
colnames = ['xeid'] + [new_names(s) for s in df.columns[1:]]

print(colnames[:10])

['xeid', 'x31_0_0', 'x21022_0_0', 'x130836_0_0', 'x130838_0_0', 'x130840_0_0', 'x130842_0_0', 'x130846_0_0', 'x130848_0_0', 'x130852_0_0']


In [12]:
df = df.toDF(*colnames)

In [14]:
df = df.drop(
    'x130836_0_0', 'x130838_0_0', 'x130842_0_0'
)  # these do not converge during Firth correction

In [19]:
df.write.csv("/tmp/phenos.tsv", sep="\t", header=True, emptyValue='NA')

In [20]:
subprocess.run(
    ["hadoop", "fs", "-rm", "/tmp/phenos.tsv/_SUCCESS"], check=True, shell=False
)
subprocess.run(
    ["hadoop", "fs", "-get", "/tmp/phenos.tsv", "../tmp/phenos.tsv"],
    check=True,
    shell=False,
)

CompletedProcess(args=['hadoop', 'fs', '-get', '/tmp/phenos.tsv', '../tmp/phenos.tsv'], returncode=0)

In [21]:
!sed -e '3,${/^xeid/d' -e '}' ../tmp/phenos.tsv/part* > ../tmp/psychiatric.raw.tsv
#!sed -i 's|""|NA|g' ../tmp/psychiatric.raw.csv

In [22]:
# Upload to project

subprocess.run(
    ["dx", "upload", "../tmp/psychiatric.raw.tsv", "--path", "Data/phenotypes/"],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '../tmp/psychiatric.raw.tsv', '--path', 'Data/phenotypes/'], returncode=0)